# Advanced NDArray tutorial: NumPy arrays and NDArrays

In the [NDArray basics tutorial](02.ndarray-basics.html) we saw how to create NDArray and set slices to the values of a NumPy array. But you can also create a NDArray from a NumPy array with the `asarray` function. Let's check this out!

In [1]:
import blosc2
import numpy as np

## Creating a NDArray from a NumPy array

We will first create a NumPy array and create a NDArray from it.

In [3]:
shape = (100, 100, 100)
dtype = np.float64
nparray = np.linspace(0, 100, np.prod(shape), dtype=dtype).reshape(shape)
b2ndarray = blosc2.asarray(nparray)
b2ndarray.info

{
    'blocksize': 262144,
    'clevel': 1,
    'codec': <Codec.ZSTD: 5>,
    'codec_meta': 0,
    'filters': [
        <Filter.NOFILTER: 0>,
        <Filter.NOFILTER: 0>,
        <Filter.NOFILTER: 0>,
        <Filter.NOFILTER: 0>,
        <Filter.NOFILTER: 0>,
        <Filter.SHUFFLE: 1>
    ],
    'filters_meta': [0, 0, 0, 0, 0, 0],
    'nthreads': 2,
    'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
    'typesize': 8,
    'use_dict': 0
}

{'nthreads': 2}

type,NDArray
shape,"(100, 100, 100)"
chunks,"(100, 100, 100)"
blocks,"(32, 32, 32)"
dtype,float64
cratio,16.00
cparams,"{'codec': , 'codec_meta': 0, 'clevel': 1, 'use_dict': 0, 'typesize': 8, 'nthreads': 2, 'blocksize': 262144, 'splitmode': , 'filters': [, , , , , ], 'filters_meta': [0, 0, 0, 0, 0, 0]}"
dparams,{'nthreads': 2}


## Building a NDArray from a buffer

Furthermore, you can create a NDArray filled with data from a buffer:

In [12]:
buffer = bytes(np.random.normal(0, 1, np.prod(shape)) * 8)
b2ndarray = blosc2.frombuffer(buffer, shape, dtype=dtype)
print("Compression ratio:", b2ndarray.schunk.cratio)
b2ndarray[:5, :5, :5]

Compression ratio: 2.3467473552048808


array([[[-1.12523103e+01,  3.53598924e+00, -6.85864571e+00,
         -7.60263662e-01, -2.64170413e+00],
        [ 4.34752602e+00,  4.08975930e+00, -5.79605827e+00,
          2.57459369e+00,  2.07877689e+00],
        [ 5.01761802e+00, -1.46191264e+00,  3.07027224e+00,
          2.90309861e+00,  4.09364713e+00],
        [-8.53101794e+00, -2.51623188e+00, -6.33870900e+00,
          6.40291284e+00,  1.05489428e+01],
        [ 7.84056332e-01, -8.56559604e+00, -7.85843083e+00,
          2.16577427e+01, -5.28259599e+00]],

       [[ 3.30303888e+00,  1.35989716e+01,  1.34718221e+01,
          8.99952277e+00, -6.20852658e+00],
        [ 7.21785744e+00, -5.84436590e+00,  6.65154783e+00,
          2.54472565e+00,  6.81276894e+00],
        [-8.96319805e+00, -1.34509384e+01,  7.42518205e-01,
         -1.52421355e+00,  6.43890261e+00],
        [ 4.84817556e+00, -9.46262794e+00, -1.43329590e+01,
         -1.01745577e+01, -6.81035506e+00],
        [ 2.53173405e+00,  5.10216816e-01, -4.31221872e+00,
  

## Serializing NumPy arrays

If what you want is to create a serialized, compressed version of a NumPy array, you can use the newer (and faster) functions to store it either in-memory or on-disk.  The specification of such a contiguous compressed representation, aka **cframe** can be seen [here](https://github.com/Blosc/c-blosc2/blob/main/README_CFRAME_FORMAT.rst).

### In-memory

For obtaining an in-memory representation, you can use `pack_tensor`. In comparison with its former version (`pack_array`), it is way faster and does not have the 2 GB size limitation:

In [13]:
shape = (2**10, ) * 3
np_array = np.arange(np.prod(shape), dtype=np.int32)  # 4 GB array

packed_arr2 = blosc2.pack_tensor(np_array)
unpacked_arr2 = blosc2.unpack_tensor(packed_arr2)

### On-disk

To store the serialized buffer on-disk you want to use `save_tensor` and `load_tensor`:

In [10]:
blosc2.save_tensor(np_array, urlpath="ondisk_array.b2frame", mode="w")
np_array2 = blosc2.load_tensor("ondisk_array.b2frame")
np.array_equal(np_array, np_array2)

True